# Load Requests (This is a subsitute for getting Requests from the User Inteface)

This notebook reads stored user Requests from a DataRepo and ingests them as nodes into your KnowledgeGraph.

* Iterate through the requests in the repository

* For each Request:
- Create (or merge) a Request node
- Link the Request node to the User node (create if necessary)
- Load known Keywords (in Ref Data) and create Entities for any that exist in the Request Node
- Run Named Entity Recognition on the Request Node text and create Entities for any found

The result of this step includes adding:
- User nodes (existing or newly created)
- Request nodes, connected to User nodes with an REQUESTED_BY relationship
- Entity nodes, connected to Request node by a MENTIONED_IN relationship


In [ ]:
import os
import logging
import re

## Parameters
OpenTLDR workflows use the notebook block tagged as "parameters" to inject variables (for example to configure special requests).

> **Do Not Change Variable Names in the Parameters Block** you are welcome to change the values of these parameter variables, but please do not change their names. They are used elsewhere in the notebook and in other workflow processes.

In [ ]:
# Notebook Specific Parameters
data_repo_config = {'repo_type': 'files', 'path': '../Data/Sample/request'}
spacy_model = "en_core_web_sm"

# Standard Parameters 

# Logging level ranges are (from least to most verbose): ERROR, WARN, INFO, DEBUG
logging_level = logging.INFO

# List of the Request Uids to Ingest
list_of_uids = None

# level of unnecessary output
verbose = True

## Setup

In [ ]:
logging.getLogger("OpenTLDR").setLevel(logging_level)

from opentldr.Domain import User, Request, Entity
from opentldr import KnowledgeGraph, DataRepo

kg=KnowledgeGraph()

## Ingest Requests from the data repo into the KG

In [ ]:
if data_repo_config is not None:
    
    repo = DataRepo(kg,data_repo_config)
    
    if verbose:
        print("Loading Requests from: {}".format(repo.describe()))
    
    list_of_uids =  repo.importData()
    
    print("Loaded {count} requests from the repository.".format(count=len(list_of_uids)))
else:
    print("No DataRepo specified for Requests.")

## Process the newly created Request nodes


### Named Entity Recognition (NER)

This is the process of detecting objects that are identified within the text.


In [ ]:
from NerWithSpacy import *
ner = NerWithSpacy(verbose=True, model_name=spacy_model)

### Keywords
One example of such an entity are the presense of keywords.

In [ ]:
keywords = []

for node in kg.get_all_reference_nodes():
    if node.type == "KEYWORD":
        keywords.append(node.text)
if verbose:
    print("Will look for {} known keywords: {}".format(len(keywords),keywords))

### Discover Entities in the Request node's text
Iterate over the imported Requests, recognize entities of interest, and add those nodes to the KG

In [ ]:
for uid in list_of_uids:
    node = kg.get_request_by_uid(uid)
    if verbose:
        print("\nProcessing Request {title}:".format(title=node.title))
    
    # avoid adding duplicate entities for the same text value
    ner.skip = [ e.text for e in kg.get_entities_by_content(node) ]

   # Iterate Keywords first
    for keyword in keywords:
        if keyword not in ner.skip:
            if re.search(keyword, node.text, re.IGNORECASE):
                entity_node=kg.add_entity(node,text=keyword, type="KEYWORD")
                ner.skip.append(entity_node.text)
                if verbose:
                    print(" - Added entity '{text}' of type {type}".format(text=entity_node.text, type=entity_node.type))
    

    # Then search for Named Entities (avoid text being both)
    for type, text in ner.process(node.text):
        kg.add_entity(node = node, text=text, type=type)
        ner.skip.append(text)
        if verbose:
            print(" - Added entity '{text}' of type {type}".format(text=text, type=type))

# Close the KG

In [ ]:
kg.close()